## 대여소 정보 전처리

대여소정보 API와 공공데이터가 최신화되어 있지 않아 크롤링된 정보롤 활용한다.

참고 : https://yahwang.github.io/posts/5

> 주소 기반으로 구단위 정보 정리

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('crawled_bike_loc_info.csv')

In [3]:
df.columns = ['CONTENT_ID', 'CONTENT_NM', 'CRADLE_COUNT', 'ADDRESS','LATITUDE', 'LONGITUDE' ]

In [4]:
df.head()

,CONTENT_ID,CONTENT_NM,CRADLE_COUNT,ADDRESS,LATITUDE,LONGITUDE
0,1001,광진교 남단 사거리(디지털프라자앞),15,서울특별시 강동구 구천면로 171 376 남단사거리,37.541805,127.124718
1,1002,해공공원(천호동),10,서울특별시 강동구 올림픽로 702 265,37.545219,127.125916
2,1003,해공도서관앞,20,서울특별시 강동구 올림픽로 702 367,37.543915,127.125458
3,1004,삼성광나루아파트 버스정류장,10,서울특별시 강동구 상암로3길 77 암사동 534,37.553349,127.128860
4,1006,롯데캐슬 115동앞,15,서울특별시 강동구 양재대로 1665 명일동 304,37.554920,127.142799


###  구 처리

In [5]:
df['ADDR_GU'] = df['ADDRESS'].map(lambda x: x.split()[1])

In [6]:
df.head()

,CONTENT_ID,CONTENT_NM,CRADLE_COUNT,ADDRESS,LATITUDE,LONGITUDE,ADDR_GU
0,1001,광진교 남단 사거리(디지털프라자앞),15,서울특별시 강동구 구천면로 171 376 남단사거리,37.541805,127.124718,강동구
1,1002,해공공원(천호동),10,서울특별시 강동구 올림픽로 702 265,37.545219,127.125916,강동구
2,1003,해공도서관앞,20,서울특별시 강동구 올림픽로 702 367,37.543915,127.125458,강동구
3,1004,삼성광나루아파트 버스정류장,10,서울특별시 강동구 상암로3길 77 암사동 534,37.553349,127.128860,강동구
4,1006,롯데캐슬 115동앞,15,서울특별시 강동구 양재대로 1665 명일동 304,37.554920,127.142799,강동구


### 주소 오류로 인한 오류

In [8]:
df[df['ADDR_GU'].map(lambda x: x[-1]) != '구'].count()

CONTENT_ID      61
CONTENT_NM      61
CRADLE_COUNT    61
ADDRESS         61
LATITUDE        61
LONGITUDE       61
ADDR_GU         61
dtype: int64

In [9]:
error_df = df.loc[ df[df['ADDR_GU'].map(lambda x: x[-1]) != '구'].index ]

In [10]:
error_df['ADDR_GU'].unique()

array(['성내동', '상일동', '명일동', '길동', '암사동', '고덕동', '강일동', '양재대로', '서울',
       '중화동', '면목동', '상봉동', '미아동', '수유동', '번동', '동일로', '창동', '가산동', '시흥동',
       '독산동', '사당동', '관악로', '서초동', '잠원동', '신원동', '양재동', '염곡동', '상암동',
       '진관동', '응암동', '녹번동', '불광로', '불광동', '증산동', '역촌동', '갈현동'],
      dtype=object)

###  오류 처리

In [11]:
addr_dict = {}
for addr in error_df['ADDR_GU'].unique():
    if addr in ['진관동','녹번동','불광동','불광로','갈현동','역촌동','응암동','증산동']:
        addr_dict[addr] = '은평구'
    if addr in ['성내동','상일동','명일동','길동','암사동','고덕동','강일동']:
        addr_dict[addr] = '강동구'
    if addr in ['중화동', '면목동', '상봉동']:
        addr_dict[addr] = '중랑구'
    if addr in ['미아동','수유동','번동']:
        addr_dict[addr] = '강북구'
    if addr in ['가산동','시흥동','독산동']:
        addr_dict[addr] = '금천구'
    if addr in ['사당동']:
        addr_dict[addr] = '동작구'
    if addr in ['서초동', '잠원동', '신원동', '양재동', '염곡동']:
        addr_dict[addr] = '서초구'
    if addr in ['상암동']:
        addr_dict[addr] = '마포구'

In [12]:
def change_addr(addr, addr_dict):
    if addr in addr_dict.keys():
        return addr_dict[addr]
    else:
        return addr

df.loc[ df[df['ADDR_GU'].map(lambda x: x[-1]) != '구'].index, 'ADDR_GU'] = error_df['ADDR_GU'].map(lambda addr: change_addr(addr, addr_dict))

#### 나머지 오류 확인 후 처리

In [14]:
df.loc[ df[df['ADDR_GU'].map(lambda x: x[-1]) != '구'].index ]

,CONTENT_ID,CONTENT_NM,CRADLE_COUNT,ADDRESS,LATITUDE,LONGITUDE,ADDR_GU
44,1044,굽은다리역,15,서울특별시 양재대로 1572 굽은다리역,37.545399,127.142601,양재대로
209,1258,가락미륭아파트 앞,10,서울특별시 서울 송파구 가락동 195-4 가락미륭아파트 앞,37.493198,127.128998,서울
463,1655,공릉1단지아파트,15,서울특별시 동일로 1127 공릉1단지아파트,37.631111,127.070160,동일로
519,1721,창동역 2번출구,12,서울특별시 창동 74-2 창동역 2번출구,37.653015,127.046997,창동
757,2160,관악구 보건소,15,서울특별시 관악로 145 관악구 보건소,37.478626,126.951294,관악로


In [15]:
df.loc[44,'ADDR_GU'] = '강동구'
df.loc[209,'ADDR_GU'] = '송파구'
df.loc[463,'ADDR_GU'] = '노원구'
df.loc[519,'ADDR_GU'] = '도봉구'
df.loc[757,'ADDR_GU'] = '관악구'

In [16]:
df[df['ADDR_GU'].map(lambda x: x[-1]) != '구']

,CONTENT_ID,CONTENT_NM,CRADLE_COUNT,ADDRESS,LATITUDE,LONGITUDE,ADDR_GU


### 주의사항 : CONTENT_ID가 없는 경우

In [28]:
df[df['CONTENT_ID']==0]

,CONTENT_ID,CONTENT_NM,CRADLE_COUNT,ADDRESS,LATITUDE,LONGITUDE,ADDR_GU
1526,0,위트콤,14,서울특별시 서초구 방배로 110 위트콤,0.0,0.0,서초구
1527,0,위트콤공장,66,서울특별시 서초구 방배로 110 석교빌딩 4층,0.0,0.0,서초구


In [31]:
df.loc[1526,'CONTENT_ID'] = 99998 # 임의 변경
df.loc[1527,'CONTENT_ID'] = 99999

In [34]:
df.to_csv('bike_info_final.csv',index=False)